In [75]:
import pandas as pd 
import os
import joblib
import numpy as np
# sklearn
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer

In [2]:
from pymongo import MongoClient
import pandas as pd

# # MongoDB connection details
mongo_url = "mongodb://6.tcp.eu.ngrok.io:19632"  # Replace with your MongoDB URL
db_name = "phones_db"            # Replace with your database name
collection_name = "phones"  # Replace with your collection name

# #Connect to MongoDB
client = MongoClient(mongo_url)
db = client[db_name]
collection = db[collection_name]

# # Fetch data from MongoDB
data = list(collection.find())

# # Close the connection
client.close()

# # Convert MongoDB data to DataFrame
# df = pd.DataFrame(data)

# # Drop MongoDB specific '_id' field if not needed
if '_id' in df.columns:
    df.drop('_id', axis=1, inplace=True)

# # Display the DataFrame
df.head()

In [3]:
df.head()

,Name,campany,Main Image,Price,model,Ratings,Number of Ratings,Secondary Camera,Primary Camera (MP),RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
0,Galaxy A05s Dual SIM Black 4GB RAM 64GB 4G LTE...,Samsung,No cover Image,5599.0,Samsung Galaxy A05s bundle,4.8,16,NaN,NaN,4.0,5000.0,64.0,Android,NaN
1,Galaxy A15 Dual Sim Blue Black 4GB RAM 128GB 4...,Samsung,No cover Image,6999.0,SM-A155FZKDMEA / SM-A156E/DS,4.4,6.4K,12 - 15.9 MP,50+5+2 MP,4.0,5000.0,128.0,Android,NaN
2,A70 Dual SIM Field Green (3+5)GB RAM 128GB 4G...,ITEL,No cover Image,3722.0,A665L,4.5,31,NaN,NaN,8.0,5000.0,128.0,NaN,NaN
3,Galaxy A15 Dual Sim Light Blue 4GB RAM 128GB 4...,Samsung,No cover Image,6666.0,SM-A155FLBDMEA,4.4,5.2K,12 - 15.9 MP,50+5+2 MP,4.0,5000.0,128.0,Android,NaN
4,Galaxy A15 Dual SIM Blue/Black 6GB RAM 128GB 4...,Samsung,No cover Image,7599.0,"SM-A155FZKGMEA, SM-A155F/DSN",4.4,1.0K,5 - 7.9 MP,50+5+2 MP,6.0,5000.0,128.0,Android,NaN


In [4]:
df.isna().sum()

Name                     0
campany                  0
Main Image               0
Price                    1
model                    0
Ratings                  0
Number of Ratings        0
Secondary Camera       344
Primary Camera (MP)    353
RAM Size                19
Battery Size            27
Internal Memory         14
Operating System       245
Processor Name         516
dtype: int64

In [5]:
df.drop(columns=['Secondary Camera','Primary Camera (MP)','model','Main Image',"Name","Ratings","Number of Ratings"],inplace=True)

In [6]:
df.head()

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
0,Samsung,5599.0,4.0,5000.0,64.0,Android,NaN
1,Samsung,6999.0,4.0,5000.0,128.0,Android,NaN
2,ITEL,3722.0,8.0,5000.0,128.0,NaN,NaN
3,Samsung,6666.0,4.0,5000.0,128.0,Android,NaN
4,Samsung,7599.0,6.0,5000.0,128.0,Android,NaN


In [7]:
df.isna().sum()

campany               0
Price                 1
RAM Size             19
Battery Size         27
Internal Memory      14
Operating System    245
Processor Name      516
dtype: int64

In [8]:
df["RAM Size"].value_counts()

RAM Size
8.0      318
4.0      205
6.0      113
12.0     105
3.0       39
2.0       25
32.0      20
16.0      14
1.0        7
48.0       5
512.0      4
0.0        1
128.0      1
256.0      1
5.0        1
Name: count, dtype: int64

In [9]:
df[df['RAM Size'].isna()]

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
143,unknown-brand,NaN,NaN,NaN,NaN,NaN,NaN
152,GIRASOLE,649.0,NaN,800.0,NaN,NaN,NaN
386,unitronics,525.0,NaN,NaN,NaN,NaN,NaN
418,GIRASOLE,680.0,NaN,800.0,NaN,NaN,NaN
437,unitronics,550.0,NaN,NaN,NaN,NaN,NaN
467,GIRASOLE,666.0,NaN,800.0,NaN,NaN,NaN
476,unitronics,530.0,NaN,800.0,32.0,NaN,NaN
483,unitronics,530.0,NaN,800.0,32.0,NaN,NaN
600,هيون,3499.0,NaN,NaN,NaN,NaN,NaN
602,unitronics,530.0,NaN,800.0,32.0,NaN,NaN


In [10]:
most_frequant_ram_size= df['RAM Size'].mode()[0]

In [11]:
df['RAM Size'].fillna(most_frequant_ram_size,inplace=True)

In [12]:
df[df['RAM Size'].isna()]

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name


In [13]:
df["Battery Size"].value_counts()

Battery Size
5000.0     490
5100.0      33
5160.0      26
800.0       22
4000.0      20
          ... 
20000.0      1
803.0        1
4600.0       1
10200.0      1
2000.0       1
Name: count, Length: 64, dtype: int64

In [14]:
df[df['Battery Size'].isna()]

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
130,Apple,21554.95,4.0,NaN,64.0,iOS,NaN
143,unknown-brand,NaN,8.0,NaN,NaN,NaN,NaN
146,Apple,20142.30,4.0,NaN,64.0,iOS,Apple Bionic
168,Lenovo,15750.00,8.0,NaN,128.0,Android,NaN
184,Apple,20142.30,4.0,NaN,64.0,iOS,Apple Bionic
215,Lenovo,22222.00,6.0,NaN,128.0,Android,NaN
218,Apple,21555.00,4.0,NaN,64.0,iOS,Apple Bionic
326,Apple,13998.95,3.0,NaN,64.0,iOS,Apple
374,Samsung,9549.00,8.0,NaN,128.0,Android,NaN
386,unitronics,525.00,8.0,NaN,NaN,NaN,NaN


In [15]:
most_frequant_baterry_size = df["Battery Size"].mode()[0]

In [16]:
df["Battery Size"].fillna(most_frequant_baterry_size,inplace=True)

In [17]:
df[df['Battery Size'].isna()]

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name


In [18]:
df.isna().sum()

campany               0
Price                 1
RAM Size              0
Battery Size          0
Internal Memory      14
Operating System    245
Processor Name      516
dtype: int64

In [19]:
df['Internal Memory'].value_counts()

Internal Memory
128.0    314
256.0    289
64.0      99
512.0     54
32.0      52
4.0       23
16.0      13
8.0        8
1.0        4
6.0        3
18.0       1
0.0        1
48.0       1
21.0       1
125.0      1
Name: count, dtype: int64

In [20]:
df[df['Internal Memory'].isna()]

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
143,unknown-brand,NaN,8.0,5000.0,NaN,NaN,NaN
152,GIRASOLE,649.0,8.0,800.0,NaN,NaN,NaN
357,KXD,399.0,32.0,1000.0,NaN,NaN,NaN
386,unitronics,525.0,8.0,5000.0,NaN,NaN,NaN
418,GIRASOLE,680.0,8.0,800.0,NaN,NaN,NaN
437,unitronics,550.0,8.0,5000.0,NaN,NaN,NaN
467,GIRASOLE,666.0,8.0,800.0,NaN,NaN,NaN
600,هيون,3499.0,8.0,5000.0,NaN,NaN,NaN
608,IKU,420.0,8.0,5000.0,NaN,NaN,NaN
632,ماركة غير محددة,480.0,8.0,5000.0,NaN,NaN,NaN


In [21]:
df['Internal Memory'].fillna(df['Internal Memory'].mode()[0],inplace=True)

In [22]:
df.isna().sum()

campany               0
Price                 1
RAM Size              0
Battery Size          0
Internal Memory       0
Operating System    245
Processor Name      516
dtype: int64

---

In [23]:
df[df["Operating System"].isna() & (df["campany"] == "IKU")]

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
283,IKU,410.0,32.0,600.0,32.0,NaN,NaN
463,IKU,570.0,32.0,5000.0,32.0,NaN,NaN
492,IKU,399.0,32.0,600.0,32.0,NaN,NaN
519,IKU,499.0,32.0,600.0,32.0,NaN,NaN
543,IKU,2449.0,2.0,4000.0,32.0,NaN,NaN
561,IKU,2449.0,2.0,4000.0,32.0,NaN,NaN
573,IKU,2449.0,2.0,4000.0,32.0,NaN,NaN
583,IKU,2449.0,2.0,4000.0,32.0,NaN,NaN
608,IKU,420.0,8.0,5000.0,128.0,NaN,NaN
620,IKU,2599.0,2.0,4000.0,16.0,NaN,NaN


In [24]:
df['campany'].value_counts()

campany
Samsung            170
Xiaomi             133
realme             104
Apple               84
OPPO                79
Infinix             46
NOKIA               39
vivo                37
Honor               36
IKU                 33
ITEL                31
HUAWEI              13
TECNO               10
unitronics          10
Lenovo               8
ACE                  6
cmf by NOTHING       4
KXD                  4
Google               4
OnePlus              4
GIRASOLE             3
Nothing              3
Generic              3
Lava                 2
benco                2
CAT                  1
Hope                 1
Alcatel              1
unknown-brand        1
هيون                 1
ماركة غير محددة      1
REDMAGIC             1
TCL                  1
ZTE                  1
HUION                1
Name: count, dtype: int64

In [25]:
df['Operating System'].value_counts()

Operating System
Android             534
iOS                  84
HarmonyOS             9
Nokia OS              2
Nokia X Software      1
Symbian               1
Linux                 1
Unix                  1
Name: count, dtype: int64

In [27]:
campanies = df['campany'].to_list()
for campany in campanies:
    if campany =="Apple":
        df.loc[df["Operating System"].isna() & (df["campany"] == campany), "Operating System"] = "iOS"
    else:
        df.loc[df["Operating System"].isna() & (df["campany"] == campany), "Operating System"] = "Android"    

In [28]:
df['Operating System'].value_counts()

Operating System
Android             779
iOS                  84
HarmonyOS             9
Nokia OS              2
Nokia X Software      1
Symbian               1
Linux                 1
Unix                  1
Name: count, dtype: int64

In [29]:
df.isna().sum()

campany               0
Price                 1
RAM Size              0
Battery Size          0
Internal Memory       0
Operating System      0
Processor Name      516
dtype: int64

In [30]:
df['Processor Name'].value_counts()

Processor Name
Qualcomm Snapdragon    91
MediaTek Helio         89
Unisoc                 52
MediaTek Dimensity     44
Apple                  39
Apple Bionic           25
Quad-Core               5
MediaTek MT Series      4
MediaTek                3
Google Tensor           3
Octa-Core               2
HiSilicon Kirin         2
Samsung Exynos          2
Snapdragon              1
Name: count, dtype: int64

In [31]:
df[df['Processor Name'].isna()]

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
0,Samsung,5599.0,4.0,5000.0,64.0,Android,NaN
1,Samsung,6999.0,4.0,5000.0,128.0,Android,NaN
2,ITEL,3722.0,8.0,5000.0,128.0,Android,NaN
3,Samsung,6666.0,4.0,5000.0,128.0,Android,NaN
4,Samsung,7599.0,6.0,5000.0,128.0,Android,NaN
...,...,...,...,...,...,...,...
871,Generic,450.0,32.0,1000.0,32.0,Android,NaN
872,Generic,450.0,32.0,1000.0,32.0,Android,NaN
875,vivo,11399.0,8.0,5000.0,256.0,Android,NaN
876,Samsung,9799.0,8.0,5000.0,256.0,Android,NaN


In [35]:
df[df['campany'] == 'Samsung'][['Processor Name']].mode(0)

,Processor Name
0,Qualcomm Snapdragon


In [36]:
for company in campanies:
    df.loc[df["Processor Name"].isna() & (df["campany"] == campany), "Processor Name"] = df[df['campany'] == campany][['Processor Name']].mode(0)
    

In [107]:
# df.loc[df["Processor Name"].isna() & (df["campany"] == "samsung"), "Processor Name"] = "Qualcomm Snapdragon"

In [37]:
df.isna().sum()

campany               0
Price                 1
RAM Size              0
Battery Size          0
Internal Memory       0
Operating System      0
Processor Name      516
dtype: int64

In [38]:
df["Processor Name"].fillna(df['Processor Name'].mode()[0],inplace=True)

In [39]:
df.isna().sum()

campany             0
Price               1
RAM Size            0
Battery Size        0
Internal Memory     0
Operating System    0
Processor Name      0
dtype: int64

In [46]:
df.dropna(axis=0)

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
0,Samsung,5599.0,4.0,5000.0,64.0,Android,Qualcomm Snapdragon
1,Samsung,6999.0,4.0,5000.0,128.0,Android,Qualcomm Snapdragon
2,ITEL,3722.0,8.0,5000.0,128.0,Android,Qualcomm Snapdragon
3,Samsung,6666.0,4.0,5000.0,128.0,Android,Qualcomm Snapdragon
4,Samsung,7599.0,6.0,5000.0,128.0,Android,Qualcomm Snapdragon
...,...,...,...,...,...,...,...
873,realme,7800.0,8.0,5000.0,256.0,Android,MediaTek Helio
874,realme,7200.0,8.0,5000.0,256.0,Android,Unisoc
875,vivo,11399.0,8.0,5000.0,256.0,Android,Qualcomm Snapdragon
876,Samsung,9799.0,8.0,5000.0,256.0,Android,Qualcomm Snapdragon


In [47]:
df.isna().sum()

campany             0
Price               1
RAM Size            0
Battery Size        0
Internal Memory     0
Operating System    0
Processor Name      0
dtype: int64

In [49]:
df

,campany,Price,RAM Size,Battery Size,Internal Memory,Operating System,Processor Name
143,unknown-brand,NaN,8.0,5000.0,128.0,Android,Qualcomm Snapdragon


In [50]:

df.drop(143, inplace=True)

In [51]:
df.isna().sum()

campany             0
Price               0
RAM Size            0
Battery Size        0
Internal Memory     0
Operating System    0
Processor Name      0
dtype: int64

-----------


In [52]:
X = df.drop('Price',axis = 1)
y = df['Price']

In [85]:
X.columns

Index(['campany', 'RAM Size', 'Battery Size', 'Internal Memory',
       'Operating System', 'Processor Name'],
      dtype='object')

In [56]:
X_train, X_test , y_train , y_test=train_test_split(X,y,shuffle=True,test_size=0.20,random_state=42)

In [55]:
print("X_train shape",X_train.shape)
print("t_train shape",y_train.shape)
print("X_test shape",X_test.shape)
print("y_test shape",y_test.shape)

X_train shape (745, 6)
t_train shape (745,)
X_test shape (132, 6)
y_test shape (132,)


In [57]:
num_cols = [col for col in X_train.columns if X_train[col].dtype in ['int32','int64','float32','float64']]
categ_cols = [col for col in X_train.columns if X_train[col].dtype not in ['int32','int64','float32','float64']]
categ_cols

['campany', 'Operating System', 'Processor Name']

In [64]:
X_test.isna().sum()

campany             0
RAM Size            0
Battery Size        0
Internal Memory     0
Operating System    0
Processor Name      0
dtype: int64

In [65]:
num_pipline = Pipeline(steps = [
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler',StandardScaler())
                ])

categ_pipline = Pipeline(steps=[
    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

all_pipeline = ColumnTransformer(transformers=[
                    ('numerical', num_pipline, num_cols),
                    ('categorical', categ_pipline, categ_cols),
                ])

X_train_final = all_pipeline.fit_transform(X_train)
X_test_final = all_pipeline.transform(X_test)

In [68]:
# Dump the pipeline locally in ARTIFACTS_FOLDER_PATH
ARTIFACTS_FOLDER_PATH = os.path.join(os.getcwd(), '..', 'artifacts')
os.makedirs(ARTIFACTS_FOLDER_PATH, exist_ok=True)

In [71]:
# Dump the pipeline locally --> Serialzation
joblib.dump(all_pipeline, os.path.join(ARTIFACTS_FOLDER_PATH, 'pipeline.pkl'))

['c:\\depi\\projects\\project tools\\notebook\\..\\artifacts\\pipeline.pkl']

# `Building ML Models`

evaluation metric : RMSE

* `Linear Regression`

In [76]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_final,y_train)
 
# predict
y_pred_train_lin = lin_reg.predict(X_train_final)
y_pred_test_lin = lin_reg.predict(X_test_final)

# Evaluate
rmse_train_lin =np.sqrt( mean_squared_error(y_train, y_pred_train_lin))
rmse_test_lin = np.sqrt(mean_squared_error(y_test, y_pred_test_lin))

print(f'RMSE for Training using LinearRegression is: {rmse_train_lin:.3f}')
print(f'RMSE for Testing using LinearRegression is: {rmse_test_lin:.3f}')



RMSE for Training using LinearRegression is: 8334.569
RMSE for Testing using LinearRegression is: 6876.509


In [77]:
forest_reg = RandomForestRegressor(n_estimators=500, max_depth=30, random_state=45)
forest_reg.fit(X_train_final,y_train)

# Predict and evaluate
y_pred_train_forest = forest_reg.predict(X_train_final)
y_pred_test_foerst = forest_reg.predict(X_test_final)

# Using fRMSE
RMSE_train_forest = np.sqrt(mean_squared_error(y_train, y_pred_train_forest))
RMSE_test_forest = np.sqrt(mean_squared_error(y_test, y_pred_test_foerst))

print(f'The RMSE using root mean squared error model for Training Dataset  {RMSE_train_forest:.3f}')
print(f'The RMSE using root mean squared error model for Testing Dataset  {RMSE_test_forest:.3f}')

The RMSE using root mean squared error model for Training Dataset  2009.733
The RMSE using root mean squared error model for Testing Dataset  2304.749


In [78]:
forest_reg = RandomForestRegressor(n_estimators=300, max_depth=15, 
                                    max_features=0.8, random_state=45)

# Params to search in
params_best_grid = {
    'n_estimators': np.arange(250, 400, 50),
    'max_depth': np.arange(10, 20, 2)
}

# GridSearchCV
grid_forest = GridSearchCV(estimator=forest_reg, param_grid=params_best_grid, cv=5, scoring='neg_mean_squared_error', verbose=4)
grid_forest.fit(X_train_final, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END max_depth=10, n_estimators=250;, score=-13679109.627 total time=   0.3s
[CV 2/5] END max_depth=10, n_estimators=250;, score=-13549788.349 total time=   0.3s
[CV 3/5] END max_depth=10, n_estimators=250;, score=-10557710.582 total time=   0.3s
[CV 4/5] END max_depth=10, n_estimators=250;, score=-14701815.571 total time=   0.3s
[CV 5/5] END max_depth=10, n_estimators=250;, score=-17903001.100 total time=   0.3s
[CV 1/5] END max_depth=10, n_estimators=300;, score=-13895644.896 total time=   0.3s
[CV 2/5] END max_depth=10, n_estimators=300;, score=-13491437.086 total time=   0.4s
[CV 3/5] END max_depth=10, n_estimators=300;, score=-10803131.630 total time=   0.3s
[CV 4/5] END max_depth=10, n_estimators=300;, score=-14819759.209 total time=   0.3s
[CV 5/5] END max_depth=10, n_estimators=300;, score=-17399289.218 total time=   0.3s
[CV 1/5] END max_depth=10, n_estimators=350;, score=-13752778.001 total time=   0.4s
[CV 

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=15, max_features=0.8,
                                             n_estimators=300,
                                             random_state=45),
             param_grid={'max_depth': array([10, 12, 14, 16, 18]),
                         'n_estimators': array([250, 300, 350])},
             scoring='neg_mean_squared_error', verbose=4)

In [79]:
grid_forest.best_score_

-13693174.537292022

In [80]:
grid_forest.best_params_

{'max_depth': 16, 'n_estimators': 350}

In [81]:
# The forest after tuning
forest_tuned = grid_forest.best_estimator_
forest_tuned.fit(X_train_final, y_train)

# Predict and evaluate
y_pred_train_forest_tuned = forest_tuned.predict(X_train_final)
y_pred_test_foerst_tuned = forest_tuned.predict(X_test_final)

# Using f1_score
score_train_forest_tuned =np.sqrt(mean_squared_error(y_train, y_pred_train_forest_tuned))
score_test_forest_tuned = np.sqrt(mean_squared_error(y_test, y_pred_test_foerst_tuned))

print(f'The RMSE using tuned RF model for Training Dataset with class_weigths is {score_train_forest_tuned:.3f}')
print(f'The RMSE using tuned RF model for Testing Dataset with class_weigths is {score_test_forest_tuned:.3f}')

The RMSE using tuned RF model for Training Dataset with class_weigths is 2013.962
The RMSE using tuned RF model for Testing Dataset with class_weigths is 2206.770


* `XGBoost`

In [82]:
scale = np.bincount(y_train)[0] / np.bincount(y_train)[1]

C:\Users\youssef gerges\AppData\Local\Temp\ipykernel_28968\2366938159.py:1: RuntimeWarning: invalid value encountered in scalar divide
  scale = np.bincount(y_train)[0] / np.bincount(y_train)[1]


In [83]:
xgb_reg = XGBRegressor(objective='reg:squarederror', n_estimators=250, max_depth=20, learning_rate=0.001)
xgb_reg.fit(X_train_final, y_train)

# Predict and evaluate
y_pred_train_xgb = xgb_reg.predict(X_train_final)
y_pred_test_xgb = xgb_reg.predict(X_test_final)

# Using f1_score
score_train_xgb=np.sqrt(mean_squared_error(y_train, y_pred_train_xgb))
score_test_xgb =np.sqrt(mean_squared_error(y_test, y_pred_test_xgb))

print(f'The RMSE using XGBoost model for Training Dataset {score_train_xgb:.3f}')
print(f'The RMSE using XGBoost model for Testing Dataset  {score_test_xgb:.3f}')

The RMSE using XGBoost model for Training Dataset 13546.604
The RMSE using XGBoost model for Testing Dataset  11043.794


In [84]:
joblib.dump(lin_reg, os.path.join(ARTIFACTS_FOLDER_PATH, 'lin_reg.pkl'), compress=9)
joblib.dump(forest_reg, os.path.join(ARTIFACTS_FOLDER_PATH, 'forest_reg.pkl'), compress=9)
joblib.dump(xgb_reg, os.path.join(ARTIFACTS_FOLDER_PATH, 'Xgboost.pkl'), compress=9)
joblib.dump(forest_tuned , os.path.join(ARTIFACTS_FOLDER_PATH, 'forest_tuned.pkl'), compress=9)


['c:\\depi\\projects\\project tools\\notebook\\..\\artifacts\\forest_tuned.pkl']

-----------

In [4]:
df = pd.read_csv("C:/Users/youssef gerges/Downloads/archive (18)/diabetes_prediction_dataset.csv")

In [5]:
df.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64